In [6]:
from keras.layers import Input, Dense, Activation, Dropout, LeakyReLU, Lambda, PReLU, LSTM, SimpleRNN
from keras.models import Model, Sequential, model_from_json
from keras.callbacks import EarlyStopping
from keras import optimizers, activations, regularizers
from keras import backend as K
from sklearn.model_selection import train_test_split, KFold
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import json

In [5]:
'''Import data'''

# File path
data = h5py.File('/Users/maggiezhang/Dropbox/AtmosphereMachineLearning/DensityData/JB2008_1999_2010_ROM_r100.mat', 'r')

# Reduced-order density data
print(data.keys())
dens_data = data.get('densityDataLogVarROM100')
dens_data = np.array(dens_data)

# Space weather data
sw_data = data.get('SWdataFull')
sw_data = np.array(sw_data)
sw_data = sw_data.transpose()
sw_col_names = ["doy", "UThrs", "F10", "F10B", "S10", "S10B", "XM10", "XM10B", "Y10", "Y10B", "DSTDTC", "GWRAS", "SUN(1)", "SUN(2)"]
df = pd.DataFrame(sw_data, columns = sw_col_names)
print(df.head())

# Only consider first 10 modes/dimensions of reduced data
dens_data = dens_data[:,0:10]

'''Training data'''
# Training input

split_15 = int(dens_data.shape[0]*0.85) # manual test split 

dens_data_train = dens_data[:split_15,:]
sw_data_train = sw_data[:split_15,:]
sw_data_train_append = sw_data[1:split_15+1,:] #looks at next sw timestep because it could be useful in prediction 
sw_data_train = np.concatenate((sw_data_train, sw_data_train_append), axis=1)
x_train = np.concatenate((dens_data_train, sw_data_train), axis=1)
y_train = dens_data[1:split_15+1,:]

'''ROM LINEAR MATRICES''' 
Om = np.append(np.transpose(dens_data_train), np.transpose(sw_data_train), axis = 0)
X2 = np.transpose(y_train)
Phi = np.matmul(X2,np.linalg.pinv(Om))

r = 10
A = Phi[:r, :r]
B = Phi[:r, r:]

'''Test data'''
# Test input
dens_data_test = dens_data[split_15:-1,:]
sw_data_test = sw_data[split_15:-1,:]
sw_data_test_append = sw_data[split_15+1:,:]
sw_data_test = np.concatenate((sw_data_test, sw_data_test_append), axis = 1)
x_test = np.concatenate((dens_data_test, sw_data_test), axis=1)
y_test = dens_data[split_15+1:,:]

<KeysViewHDF5 ['D100', 'SWdataFull', 'U100', 'altitudes', 'densityDataLogVarROM100', 'densityDataMeanLog', 'latitudes', 'localSolarTimes']>
    doy  UThrs    F10   F10B    S10   S10B   XM10  XM10B    Y10   Y10B  \
0  75.0   12.0  150.1  129.7  150.5  137.6  158.1  130.7  148.8  138.7   
1  75.0   13.0  150.1  129.7  150.5  137.6  158.1  130.7  148.8  138.7   
2  75.0   14.0  150.1  129.7  150.5  137.6  158.1  130.7  148.8  138.7   
3  75.0   15.0  150.1  129.7  150.5  137.6  158.1  130.7  148.8  138.7   
4  75.0   16.0  150.1  129.7  150.5  137.6  158.1  130.7  148.8  138.7   

   DSTDTC     GWRAS    SUN(1)    SUN(2)  
0    17.5  6.172181 -0.072585 -0.031433  
1    38.5  0.151512 -0.071919 -0.031146  
2    38.5  0.414028 -0.071254 -0.030859  
3    38.5  0.676544 -0.070589 -0.030571  
4    38.5  0.939060 -0.069923 -0.030284  


In [4]:
'''inputs: A 3D tensor with shape [batch, timesteps, feature]'''
#10 features
print(y_test.shape)
print(y_train.shape)
print(x_test.shape)
print(x_train.shape)

(15510, 10)
(87893, 10)
(15510, 38)
(87893, 38)


In [ ]:
model = Sequential()

